# Loading data to Supabase Posgres Database

We have decided to try and host the database using Supabase as the backend. Hosting this on a Supabase's cloud, we could more easily scale and build applications that can access that database. We will eventually try this with the isochrone project.

In [1]:
import pandas as pd
import os
from supabase import create_client, Client

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Supabase API

We have created a database on Supabase off-code. This was created using my(nabilersyad) account under that organization. The database mirrors the SQLite database.
Details on the created database below.

SUPABASE_URL = 'saved locally in the shell. Get the details from supabase dashboard'  
SUPABASE_KEY = 'saved in the shell. Get the details from supabase dashboard'

database: transit-stations-database  
tables: 1. stations  
        2. entrances  
        3. station_entrances  

In [2]:
#declaring the supabase client we will working with
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [3]:
#signin a user
#sign_in_response = supabase.auth.sign_in_with_password({"email": "nabile95@hotmail.com", "password": ""})


In [4]:

#user = supabase.auth.get_user()


## Stations table

Combining the various CSVs of different cities into one table and we will use this table to update data in the supabase database. Assume all data in this combined table to be the most updated

In [5]:

# Define the directory of cleansed data to transfer to sql
cleansed_data_directory = 'data_cleansed'
cleansed_kl_file = 'klang_valley_stations_cleansed.csv'
cleansed_montreal_file = 'montreal_stations_cleansed.csv'
cleansed_singapore_file = 'singapore_stations_cleansed.csv'
cleansed_combined_file = 'combined_stations_cleansed.csv'

# read cleaned dataframes
kl_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_kl_file))
montreal_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_montreal_file))
singapore_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_singapore_file))


# Combine all the dataframes
stations_data_local = pd.concat([kl_data, montreal_data, singapore_data], axis=0, ignore_index=True)

stations_data_local.index.name = 'station_id'
stations_data_local.to_csv(os.path.join(cleansed_data_directory, cleansed_combined_file), index=True)


# Print out the combined dataframe
stations_data_local


,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,
0,KL Sentral,KA01,Keretapi Tanah Melayu,3.134603,101.686567,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,Outram Park MRT Station,TE17,NaN,1.280400,103.840100,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN
532,Maxwell MRT Station,TE18,NaN,1.280600,103.844000,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN
533,Shenton Way MRT Station,TE19,NaN,1.277540,103.850770,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN


We will now retrieve the equivalent table in supabase. This will be the table we need to update

In [6]:
response = supabase.table('stations').select("*").execute()
data,_ = response
stations_data_supa = pd.DataFrame(data[1])
stations_data_supa.set_index('station_id',inplace=True)
stations_data_supa

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
5,Seputeh,KB02,Keretapi Tanah Melayu,3.113697,101.681299,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,Maxwell MRT Station,TE18,None,1.280600,103.844000,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None
533,Shenton Way MRT Station,TE19,None,1.277540,103.850770,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None
534,Marina Bay MRT Station,TE20,None,1.275290,103.854810,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None


### Merging local and supabase tables (Stations)

We will merge the two databases on their index in a union merge while creating  a new column to indicate for each row, which table has it

There will be 4 cases

Case 1: Row is in both local and supa table and is identical

Case 2: Row is in both tables but has some differences in one or more columns

Case 3: rows in local but not in supa - only_local

Case 4: rows not in local but in supa - only_supa


In [7]:
# Perform an outer join on the dataframes
merged_stations_data = pd.merge(stations_data_local, stations_data_supa, how='outer',left_index=True,right_index=True,indicator=True)

# Case 1 and 2: rows exist in both dataframes
both_stations_data = merged_stations_data[merged_stations_data['_merge'] == 'both']
both_stations_data = both_stations_data.drop(columns=['_merge'])

merged_stations_data['_merge'].value_counts()



both          536
left_only       0
right_only      0
Name: _merge, dtype: int64

#### Case 1 handling
We will identify the rows that are in both tables and identical from the both_data table using the 'both' indicator.

This is the extend of work that needs to be done. The supabase table does not need to be updated

In [8]:
#Case 1 
identical_stations_rows_mask = [stations_data_local.loc[i, stations_data_local.columns].equals(stations_data_supa.loc[i, stations_data_supa.columns])
                        for i in both_stations_data.index]
identical_stations_rows = both_stations_data[identical_stations_rows_mask]
identical_stations_rows = stations_data_local.loc[identical_stations_rows.index]
identical_stations_rows

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,
0,KL Sentral,KA01,Keretapi Tanah Melayu,3.134603,101.686567,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,Outram Park MRT Station,TE17,NaN,1.280400,103.840100,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN
532,Maxwell MRT Station,TE18,NaN,1.280600,103.844000,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN
533,Shenton Way MRT Station,TE19,NaN,1.277540,103.850770,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN


#### Case 2 handling
Using the mask from handling case 1, we can get the rows with different values by getting the inverse of the identical_rows_mask to get right index. Then using this index to select the relevant rows from both_data

We will then use the rows from different_rows to update the supabase table using the update() function

In [9]:
different_stations_rows_mask= [not b for b in identical_stations_rows_mask]
different_stations_rows = both_stations_data[different_stations_rows_mask]

# Replace all NaN values in the DataFrame with None
different_stations_rows = different_stations_rows.where(pd.notna(different_stations_rows), None)
different_stations_rows = stations_data_local.loc[different_stations_rows.index]

different_stations_rows

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,


In [10]:
#the following is to convert index in pandas to be a column, usable for posgres
different_stations_rows = different_stations_rows.reset_index(drop=False)

# For the different rows, you want to update data_supabase with data from data_local
for index, row in different_stations_rows.iterrows():
    # Use Supabase update method
    # Note: replace 'id' and 'your_table' with your actual id column name and table name
    data, error = supabase.table('stations').update(row.to_dict()).eq('station_id', row['station_id']).execute()


#### Case 3 handling
Using the left_only indicator, we can get the rows that are in local that needs to be inserted in the supabase table

We will then use the rows from only_local to update the supabase table using the insert() function

In [11]:

# Case 3: rows in data_local but not in data_supabase

only_local_stations = merged_stations_data[merged_stations_data['_merge'] == 'left_only']
only_local_stations = only_local_stations.drop(columns=['_merge'])

only_local_stations = stations_data_local.loc[only_local_stations.index]
only_local_stations = only_local_stations.where(pd.notna(only_local_stations), None)
only_local_stations = only_local_stations.reset_index(drop=False)

only_local_stations

,station_id,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened


In [12]:
# For these rows, you want to insert into data_supabase
data, error = supabase.table('stations').insert(only_local_stations.to_dict('records')).execute()

#### Case 4 handling
Using the right_only indicator, we can get the rows that are in supabase that needs to be removed from the supabase table

We will then use the rows from only_local to update the supabase table using the delete() function

In [13]:
# Case 4: rows not in data_local but in data_supabase
only_supabase_stations = merged_stations_data[merged_stations_data['_merge'] == 'right_only']
only_supabase_stations = only_supabase_stations.drop(columns=['_merge'])

only_supabase_stations = stations_data_local.loc[only_supabase_stations.index]
only_supabase_stations = only_supabase_stations.where(pd.notna(only_supabase_stations), None)
only_supabase_stations = only_supabase_stations.reset_index(drop=False)
only_supabase_stations

,station_id,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened


In [14]:
# For these rows, you want to delete from data_supabase
for index, row in only_supabase_stations.iterrows():
    # Use Supabase delete method
    data, error = supabase.table('stations').delete().eq('station_id', row['station_id']).execute()

## Entrances tables

There are two entrances tables that we need to create. We'll call them entrances and station_entrances.
Again assume all local data to be the most updated

We will start with entrances table

In [15]:
# Retrieve entrances table in local csv file
cleansed_entrances = 'klang_valley_entrances_cleansed.csv'

# read cleaned entrances dataframes
entrances_data_local = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_entrances))

#ensure NAs are in a posgres readable format
entrances_data_local = entrances_data_local.where(pd.notna(entrances_data_local), None)

#ensure index to be entrance_id because it is easier to work with
entrances_data_local.set_index('entrance_id',inplace=True)
entrances_data_local = entrances_data_local.sort_values(by='entrance_id')

entrances_data_local

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
1544031348,101.711374,3.145929,None,B
1631412559,101.604952,3.113222,None,None
2278515570,101.644077,3.050650,None,None
2686635178,101.699182,3.138565,None,C
3308608988,101.712717,3.158762,None,None
...,...,...,...,...
11390855314,102.094357,2.592846,None,None
11435336038,101.701394,3.144743,None,None
11459346120,101.663365,3.061779,PPR Muhibbah Blok E dan F,None


In [16]:
# Retrieve entrances table from supabase

response = supabase.table('entrances').select("*").execute()
data,_ = response
entrances_data_supa = pd.DataFrame(data[1])

#ensure index to be entrance_id because it is easier to work with
entrances_data_supa.set_index('entrance_id',inplace=True)
entrances_data_supa = entrances_data_supa.sort_values(by='entrance_id')

entrances_data_supa

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
1544031348,101.711374,3.145929,None,B
1631412559,101.604952,3.113222,None,None
2278515570,101.644077,3.050650,None,None
2686635178,101.699182,3.138565,None,C
3308608988,101.712717,3.158762,None,None
...,...,...,...,...
11250847387,101.675866,3.204803,None,None
11252711594,101.675460,3.204839,None,None
11257782484,101.696485,3.150072,None,None


### Merging local and supabase tables

We will merge the two databases on their index in a union merge while creating  a new column to indicate for each row, which table has it

There will be 4 cases

Case 1: Row is in both local and supa table and is identical

Case 2: Row is in both tables but has some differences in one or more columns

Case 3: rows in local but not in supa - only_local

Case 4: rows not in local but in supa - only_supa

In [17]:
# Perform an outer join on the dataframes
merged_data = pd.merge(entrances_data_local,entrances_data_supa,how='outer',left_index=True,right_index=True,indicator=True)

# Case 1 and 2: rows exist in both dataframes
both_data = merged_data[merged_data['_merge'] == 'both']
both_data = both_data.drop(columns=['_merge'])

merged_data['_merge'].value_counts()


both          244
left_only      17
right_only      1
Name: _merge, dtype: int64

#### Case 1 handling
We will identify the rows that are in both tables and identical from the both_data table using the 'both' indicator.

This is the extend of work that needs to be done. The supabase table does not need to be updated

In [18]:
#Case 1 
identical_rows_mask = [entrances_data_local.loc[i, entrances_data_local.columns].equals(entrances_data_supa.loc[i, entrances_data_supa.columns])
                        for i in both_data.index]
identical_rows = both_data[identical_rows_mask]
identical_rows = entrances_data_local.loc[identical_rows.index]
identical_rows

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
1544031348,101.711374,3.145929,None,B
1631412559,101.604952,3.113222,None,None
2278515570,101.644077,3.050650,None,None
2686635178,101.699182,3.138565,None,C
3308608988,101.712717,3.158762,None,None
...,...,...,...,...
11250847387,101.675866,3.204803,None,None
11252711594,101.675460,3.204839,None,None
11257782484,101.696485,3.150072,None,None


#### Case 2 handling
Using the mask from handling case 1, we can get the rows with different values by getting the inverse of the identical_rows_mask to get right index. Then using this index to select the relevant rows from both_data

We will then use the rows from different_rows to update the supabase table using the update() function

In [19]:
different_rows_mask= [not b for b in identical_rows_mask]
different_rows = both_data[different_rows_mask]

# Replace all NaN values in the DataFrame with None
different_rows = different_rows.where(pd.notna(different_rows), None)
different_rows = entrances_data_local.loc[different_rows.index]

different_rows

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
5044809585,101.719763,3.143136,None,A
5385396790,101.665646,3.150649,Jalan Damansara Endah,A
5469085436,101.740219,3.090602,Taman Mutiara;Jalan Mutiara Raya,A
5469085437,101.740646,3.091482,Taman Segar,B
5469085653,101.773094,3.041281,Jalan SS 2/2,A
5484250431,101.662551,3.143553,Jalan Johar,B
5484250432,101.663167,3.143452,Jalan Maarof;HELP Residen,A
5484250467,101.642764,3.128354,Jalan 16/11;Pusat Dagangan Phileo Damansara;Ea...,A
6375602764,101.711660,3.146021,None,E


In [20]:
#the following is to convert index in pandas to be a column, usable for posgres
different_rows = different_rows.reset_index(drop=False)

# For the different rows, you want to update data_supabase with data from data_local
for index, row in different_rows.iterrows():
    # Use Supabase update method
    # Note: replace 'id' and 'your_table' with your actual id column name and table name
    data, error = supabase.table('entrances').update(row.to_dict()).eq('entrance_id', row['entrance_id']).execute()


#### Case 3 handling
Using the left_only indicator, we can get the rows that are in local that needs to be inserted in the supabase table

We will then use the rows from only_local to update the supabase table using the insert() function

In [21]:

# Case 3: rows in data_local but not in data_supabase

only_local = merged_data[merged_data['_merge'] == 'left_only']
only_local = only_local.drop(columns=['_merge'])

only_local = entrances_data_local.loc[only_local.index]
only_local = only_local.where(pd.notna(only_local), None)
only_local = only_local.reset_index(drop=False)

only_local

,entrance_id,longitude,latitude,entrance_destination,entrance_name
0,3948655246,101.721651,3.219798,None,None
1,10271036606,101.718064,3.156989,The Troika;Masjid Asy-Syakirin;Petronas Twin T...,C
2,11307838640,101.688936,3.182278,None,None
3,11307838642,101.689163,3.182947,None,None
4,11307838648,101.689170,3.182597,None,None
5,11307838662,101.688475,3.182796,None,None
6,11348181710,101.691161,3.165027,None,None
7,11353407735,101.724172,3.164611,None,None
8,11375596033,102.226439,2.463841,None,None
9,11377990463,101.605726,3.007720,None,B


In [22]:
# For these rows, you want to insert into data_supabase
data, error = supabase.table('entrances').insert(only_local.to_dict('records')).execute()

#### Case 4 handling
Using the right_only indicator, we can get the rows that are in supabase that needs to be removed from the supabase table

We will then use the rows from only_local to update the supabase table using the delete() function

In [23]:
# Case 4: rows not in data_local but in data_supabase
only_supabase = merged_data[merged_data['_merge'] == 'right_only']
only_supabase = only_supabase.drop(columns=['_merge'])

only_supabase = entrances_data_supa.loc[only_supabase.index]
only_supabase = only_supabase.where(pd.notna(only_supabase), None)
only_supabase = only_supabase.reset_index(drop=False)
only_supabase

,entrance_id,longitude,latitude,entrance_destination,entrance_name
0,5759847421,101.663256,3.144283,None,KL2333


In [24]:
# For these rows, you want to delete from data_supabase
for index, row in only_supabase.iterrows():
    # Use Supabase delete method
    data, error = supabase.table('entrances').delete().eq('entrance_id', row['entrance_id']).execute()

## Station Entrances tables

This table tracks the relationship between station and entrances

In [25]:
# Retrieve station entrances table in local csv file
cleansed_station_entrances= 'klang_valley_stations_entrances_relation_cleansed.csv'

# read cleaned statuion entrances dataframes
station_entrances_data_local = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_station_entrances))

#ensure NAs are in a posgres readable format
station_entrances_data_local = station_entrances_data_local.where(pd.notna(station_entrances_data_local), None)

#ensure index to be entrance_id because it is easier to work with
station_entrances_data_local.set_index('relationship_id',inplace=True)
station_entrances_data_local = station_entrances_data_local.sort_values(by='entrance_id')

station_entrances_data_local


,entrance_id,station_name,station_code
relationship_id,,,
167,1544031348,Bukit Bintang (Monorail),MR6
123,1631412559,Kelana Jaya,KJ24
114,2278515570,Kinrara BK 5,SP22
213,2686635178,Maharajalela,MR3
43,3308608988,KLCC,KJ10
...,...,...,...
217,11390855314,Rembau,KB16
1,11435336038,Plaza Rakyat,SP8
0,11435336038,Plaza Rakyat,AG8


In [26]:
# Retrieve entrances table from supabase

response = supabase.table('station_entrances').select("*").execute()
data,_ = response
station_entrances_data_supa = pd.DataFrame(data[1])

#ensure index to be entrance_id because it is easier to work with
station_entrances_data_supa.set_index('relationship_id',inplace=True)
station_entrances_data_supa = station_entrances_data_supa.sort_values(by='relationship_id')

station_entrances_data_supa

,entrance_id,station_name,station_code
relationship_id,,,
0,10796851698,Pudu,AG10
1,10796851698,Pudu,SP10
2,5485710279,Kampung Baru,KJ11
3,5485710278,Kampung Baru,KJ11
4,9740843587,Masjid Jamek (KJ),KJ13
...,...,...,...
267,10839997852,Cyberjaya Utara,PY39
268,10658294223,Cyberjaya City Centre,PY30
269,10722980582,Putrajaya Sentral (MRT),PY41


In [27]:
station_entrances_data_local[station_entrances_data_local['station_code']=='KC02']

,entrance_id,station_name,station_code
relationship_id,,,
223,11264561181,Batu Kentonmen,KC02


In [28]:
station_entrances_data_supa[station_entrances_data_supa['station_code']=='KC02']

,entrance_id,station_name,station_code
relationship_id,,,


### Merging local and supabase tables

We will merge the two databases on their index in a union merge while creating  a new column to indicate for each row, which table has it

There will be 4 cases

Case 1: Row is in both local and supa table and is identical

Case 2: Row is in both tables but has some differences in one or more columns

Case 3: rows in local but not in supa - only_local

Case 4: rows not in local but in supa - only_supa

In [29]:
# Perform an outer join on the dataframes
merged_station_entrances_data = pd.merge(station_entrances_data_local,station_entrances_data_supa,how='outer',left_index=True,right_index=True,indicator=True)

# Case 1 and 2: rows exist in both dataframes
both_station_entrances_data = merged_station_entrances_data[merged_station_entrances_data['_merge'] == 'both']
both_station_entrances_data = both_station_entrances_data.drop(columns=['_merge'])

merged_station_entrances_data['_merge'].value_counts()


both          272
left_only      25
right_only      0
Name: _merge, dtype: int64

#### Case 1 handling
We will identify the rows that are in both tables and identical from the both_data table using the 'both' indicator.

This is the extend of work that needs to be done. The supabase table does not need to be updated

In [30]:
#Case 1 
identical_station_entrances_rows_mask = [station_entrances_data_local.loc[i, station_entrances_data_local.columns].equals(station_entrances_data_supa.loc[i, station_entrances_data_supa.columns])
                        for i in both_station_entrances_data.index]
identical_station_entrances_rows = both_station_entrances_data[identical_station_entrances_rows_mask]
identical_station_entrances_rows = station_entrances_data_local.loc[identical_station_entrances_rows.index]
identical_station_entrances_rows

,entrance_id,station_name,station_code
relationship_id,,,


#### Case 2 handling
Using the mask from handling case 1, we can get the rows with different values by getting the inverse of the identical_rows_mask to get right index. Then using this index to select the relevant rows from both_data

We will then use the rows from different_rows to update the supabase table using the update() function

In [31]:
different_station_entrances_rows_mask= [not b for b in identical_station_entrances_rows_mask]
different_station_entrances_rows = both_station_entrances_data[different_station_entrances_rows_mask]

# Replace all NaN values in the DataFrame with None
different_station_entrances_rows = different_station_entrances_rows.where(pd.notna(different_station_entrances_rows), None)
different_station_entrances_rows = station_entrances_data_local.loc[different_station_entrances_rows.index]

different_station_entrances_rows

,entrance_id,station_name,station_code
relationship_id,,,
0,11435336038,Plaza Rakyat,AG8
1,11435336038,Plaza Rakyat,SP8
2,10796851698,Pudu,AG10
3,10796851698,Pudu,SP10
4,5485710279,Kampung Baru,KJ11
...,...,...,...
267,10270868726,Hospital Kuala Lumpur,PY18
268,10270868727,Hospital Kuala Lumpur,PY18
269,10271036579,Raja Uda,PY19


#### Case 3 handling
Using the left_only indicator, we can get the rows that are in local that needs to be inserted in the supabase table

We will then use the rows from only_local to update the supabase table using the insert() function

In [32]:

# Case 3: rows in data_local but not in data_supabase

only_local_station_entrances = merged_station_entrances_data[merged_station_entrances_data['_merge'] == 'left_only']
only_local_station_entrances = only_local_station_entrances.drop(columns=['_merge'])

only_local_station_entrances = station_entrances_data_local.loc[only_local_station_entrances.index]
only_local_station_entrances = only_local_station_entrances.where(pd.notna(only_local_station_entrances), None)
only_local_station_entrances = only_local_station_entrances.reset_index(drop=False)

only_local_station_entrances

,relationship_id,entrance_id,station_name,station_code
0,272,10271036605,Ampang Park (MRT),PY20
1,273,9900002870,Ampang Park (MRT),PY20
2,274,10216200615,Persiaran KLCC,PY21
3,275,10271036606,Persiaran KLCC,PY21
4,276,10271039842,Conlay,PY22
5,277,10271039830,Conlay,PY22
6,278,9924537197,Chan Sow Lin (MRT),PY24
7,279,10271036984,Chan Sow Lin (MRT),PY24
8,280,10271071855,Chan Sow Lin (MRT),PY24
9,281,10271036965,Chan Sow Lin (MRT),PY24


In [33]:
# For these rows, you want to insert into data_supabase
data, error = supabase.table('station_entrances').insert(only_local_station_entrances.to_dict('records')).execute()

#### Case 4 handling
Using the right_only indicator, we can get the rows that are in supabase that needs to be removed from the supabase table

We will then use the rows from only_local to update the supabase table using the delete() function

In [34]:
# Case 4: rows not in data_local but in data_supabase
only_supabase_station_entrances = merged_station_entrances_data[merged_station_entrances_data['_merge'] == 'right_only']
only_supabase_station_entrances = only_supabase_station_entrances.drop(columns=['_merge'])

only_supabase_station_entrances = station_entrances_data_local.loc[only_supabase_station_entrances.index]
only_supabase_station_entrances = only_supabase_station_entrances.where(pd.notna(only_supabase_station_entrances), None)
only_supabase_station_entrances = only_supabase_station_entrances.reset_index(drop=False)
only_supabase_station_entrances

,relationship_id,entrance_id,station_name,station_code


In [35]:
# For these rows, you want to delete from data_supabase
for index, row in only_supabase_station_entrances.iterrows():
    # Use Supabase delete method
    data, error = supabase.table('station_entrances').delete().eq('relationship_id', row['relationship_id']).execute()